In [1]:
import torch

import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModel, BatchEncoding

## Данные

In [2]:
train_df = pd.read_csv('../russe-wsi-kit/data/main/bts-rnc/train.csv', delimiter='\t')

In [3]:
train_df['positions'] = train_df['positions'].apply(lambda x: x.split('-'))

In [4]:
train_df['word_form'] = train_df.apply(lambda x: x['context'][int(x['positions'][0]): int(x['positions'][1])+1], axis=1)

In [5]:
train_df

,context_id,word,gold_sense_id,predict_sense_id,positions,context,word_form
0,1,балка,1,NaN,"[90, 94]","маленькой комнаты. Он был очень высок, наклони...",балку
1,2,балка,1,NaN,"[69, 73]",Пантюхин в Склифе сейчас. Он выползти на улицу...,балка
2,3,балка,1,NaN,"[115, 121]",равнозначно обеспечивает и меланхоличную езду....,балками
3,4,балка,1,NaN,"[85, 90]","верхняя часть закрыта, замкнута, многократно о...",балкой
4,5,балка,1,NaN,"[66, 70]","по телевизору: наши гол забили, я вскочил от р...",балку
...,...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,"[81, 86]","дурак, но партию свою отрабатывал точно, по ус...",штампы
3487,3488,штамп,4,NaN,"[85, 90]","дело... получается, мыслить и выражать свое мн...",штампу
3488,3489,штамп,4,NaN,"[71, 77]",", что актер должен иметь пять штампов-клише ""п...",штампов
3489,3490,штамп,4,NaN,"[107, 112]",сегодняшний день в системе негосударственного ...,штампы


## BERT

In [6]:
def get_word_location(target, tokens):
    current = ''
    current_indices = []
    for i, token in enumerate(tokens):
        if token[:2] == '##':
            current += token[2:]
            current_indices.append(i)
        else:
            current = token
            current_indices = [i]
        if i < len(tokens)-1:
            next = tokens[i+1]
            if next[:2] != '##':
                if current == target:
                    return current_indices
        else:
            if current == target:
                return current_indices
    return 'not found'

In [7]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
train_df['tokens'] = train_df['context'].apply(tokenizer.tokenize)

In [9]:
train_df['word_location'] = train_df.apply(lambda x: get_word_location(x['word_form'], x['tokens']), axis=1)

In [10]:
len(set(train_df[train_df['word_location'] == 'not found']['word']))

0

In [11]:
train_df

,context_id,word,gold_sense_id,predict_sense_id,positions,context,word_form,tokens,word_location
0,1,балка,1,NaN,"[90, 94]","маленькой комнаты. Он был очень высок, наклони...",балку,"[мал, ##ень, ##кой, ко, ##мна, ##ты, ., Он, бы...","[31, 32]"
1,2,балка,1,NaN,"[69, 73]",Пантюхин в Склифе сейчас. Он выползти на улицу...,балка,"[Па, ##нт, ##ю, ##хин, в, С, ##кли, ##фе, сейч...","[29, 30]"
2,3,балка,1,NaN,"[115, 121]",равнозначно обеспечивает и меланхоличную езду....,балками,"[равно, ##зна, ##чно, обе, ##с, ##пе, ##чивает...","[35, 36, 37]"
3,4,балка,1,NaN,"[85, 90]","верхняя часть закрыта, замкнута, многократно о...",балкой,"[в, ##ерхня, ##я, часть, за, ##крыта, ,, за, #...","[29, 30, 31]"
4,5,балка,1,NaN,"[66, 70]","по телевизору: наши гол забили, я вскочил от р...",балку,"[по, теле, ##ви, ##зор, ##у, :, наши, гол, заб...","[23, 24]"
...,...,...,...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,"[81, 86]","дурак, но партию свою отрабатывал точно, по ус...",штампы,"[ду, ##рак, ,, но, партию, свою, от, ##ра, ##б...","[27, 28, 29]"
3487,3488,штамп,4,NaN,"[85, 90]","дело... получается, мыслить и выражать свое мн...",штампу,"[дело, ., ., ., получает, ##ся, ,, мы, ##сли, ...","[30, 31, 32]"
3488,3489,штамп,4,NaN,"[71, 77]",", что актер должен иметь пять штампов-клише ""п...",штампов,"[,, что, акт, ##ер, должен, иметь, пять, ш, ##...","[7, 8, 9]"
3489,3490,штамп,4,NaN,"[107, 112]",сегодняшний день в системе негосударственного ...,штампы,"[сегодня, ##шний, день, в, системе, него, ##су...","[25, 26, 27]"


In [12]:
def get_embedding(tokens, location):
    

NameError: name 'embeddings' is not defined

In [ ]:
train_df['embedding'] = train_df.apply(lambda x: get_embedding(x['tokens'], x['word_location'], axis=1)